<font color="white">.</font> | <font color="white">.</font> | <font color="white">.</font>
-- | -- | --
![NASA](http://www.nasa.gov/sites/all/themes/custom/nasatwo/images/nasa-logo.svg) | <h1><font size="+3">ASTG Python Courses</font></h1> | ![NASA](https://www.nccs.nasa.gov/sites/default/files/NCCS_Logo_0.png)

---

<CENTER>
<H1 style="color:red">
Accelerating (Numba) and Scaling (Dask) Python Codes on GPUs
</H1>
</CENTER>

## <font color='red'>What will be Covered?</font>

* What is Numba?
* How Does Numba Work?
* Numpy and Numba
* How to Use Numba?
* Parallelization with Numba
* Numba and Pandas
* Using Numba on GPUs

## <font color='red'>Reference Documents</font>

- <a href="https://nyu-cds.github.io/python-numba/05-cuda/">Introduction to Numba: CUDA Programming</a>
- <a href="https://people.duke.edu/~ccc14/sta-663/CUDAPython.html">Massively parallel programming with GPUs</a>
- <a href="https://www.kdnuggets.com/2019/07/accelerate-data-science-on-gpu.html">Here’s how you can accelerate your Data Science on GPU</a>
- 
- 
- <a href="http://numba.pydata.org/">Numba: A High Performance Python Compiler</a>
- <a href="https://www.youtube.com/watch?v=UaFSnaYh2b8">Understanding Numba - the Python and Numpy Compiler</a> (video)
- <a href="https://examples.dask.org/applications/stencils-with-numba.html">Stencil Computations with Numba</a>
- <a href="http://deepdata.com.pl/numba.html">Python on steroids - speeding up calculations with numba</a>
- <a href="https://colab.research.google.com/github/evaneschneider/parallel-programming/blob/master/COMPASS_gpu_intro.ipynb">Introduction to GPU programming with Numba</a>
- <a href="https://thedatafrog.com/en/articles/make-python-fast-numba/">Make python fast with numba</a>
- <a href="https://www.deeplearningwizard.com/deep_learning/production_pytorch/speed_optimization_basics_numba/">Speed Optimization Basics: Numba</a>
- <a href="https://murillogroupmsu.com/numba-versus-c/">High-Performance Python: Why?</a>
- <a href="https://flothesof.github.io/optimizing-python-code-numpy-cython-pythran-numba.html">Optimizing your code with NumPy, Cython, pythran and numba </a>
- <a href="https://www.polymorphe.org/index.php/looping-over-pandas-data-mkd">Looping over Pandas data</a>

## <font color='red'>What is Numba?</font>

> Numba is an open-source JIT compiler that translates a subset of Python and NumPy into fast machine code using `LLVM` (low-level virtual machine), via the llvmlite Python package. It offers a range of options for parallelising Python code for CPUs and GPUs, often with only minor code changes. 
>
>Wikipedia

The diagram below, shows all the steps carried out by Numba to execute `do_math`. 

![fig_numba](https://miro.medium.com/max/1400/1*S0S4QUjR-BsdTICtT9797Q.png)
Image Source: Continuum Analytics

- **IR**: Intermediate Representations
- **Bytecode Analysis**: Intermediate code more abstract than machine code
- **LLVM**: Low Level Virtual Machine, infrastructure to develop compilers
- **NVVM**: It is an IR compiler based on LLVM, it is designed to represent GPU kernels


    
**Usage**:

- Numba provides several utilities for code generation.
- Its central feature is the `numba.jit()` decorator. 
- Using this decorator, you can mark a function for optimization by Numba’s JIT compiler. - - - Various invocation modes trigger differing compilation options and behaviours.


In [1]:
import time
import numpy as np
import numba as nb
from numba import jit
from numba import njit
from numba import prange

**Checking your System**

The `numba -s` or `numba --sysinfo` command prints a lot of information about your system and your Numba installation and relevant dependencies.

In [ ]:
!numba -s

## <font color="red"> GPUs</font>

![GPUs](http://www.nvidia.com/docs/IO/143716/how-gpu-acceleration-works.png)
Image Source: NVIDIA

- Graphics Processing Units (GPUs) are custom designed to be very efficient at handling computer graphics and image processing.
- Central Processing Units (CPUs) handle computations serially, meaning the logic in handled in one stream: the next task will complete when the subsequent task has finished. CPUs can execute tasks in parallel across cores. For example, most computer CPUs tend to have either two, four or six cores.
- In comparison, GPUs have hundreds of 'cores'. This massively parallel architecture is what gives the GPU its high compute performance.

**Useful Terminology**

| Term | Meaning |
| ---  | --- |
| `host` | the CPU |
| `device` | the GPU |
| `host memory` | the system main memory |
| `device memory` | onboard memory on a GPU card |
| `kernels` | a GPU function launched by the host and executed on the device |
| `device function` | a GPU function executed on the device which can only be called from the device  |



**Numba and GPUs**

- Numba supports CUDA GPU programming by directly compiling a restricted subset of Python code into CUDA kernels and device functions following the CUDA execution model. 
- Kernels written in Numba appear to have direct access to NumPy arrays. 
- NumPy arrays are transferred between the CPU and the GPU automatically.

**Dask and GPUs**

- Dask can help to scale out large array and dataframe computations by combining the Dask Array and DataFrame collections with a GPU-accelerated array or dataframe library.
- The RAPIDS libraries provide a GPU accelerated Pandas-like library, `cuDF`, which interoperates well and is tested against Dask DataFrame.
- You can convert a Pandas-backed Dask DataFrame to a cuDF-backed Dask DataFrame.

![rapids](https://pbs.twimg.com/media/D2CeyaYVAAAe3kM.jpg)
Image Source: NVIDIA

#### Accessing the GPU on Google Colab

In order to access GPUs for free:

1. Go to the `Runtime` menu,
2. Click on `Change runtime type`, and 
3. In the pop-up box, under `Hardware accelerator`, select `GPU` and click on `SAVE`.

# Environment Sanity Check #

Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

Check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4.

In [ ]:
!nvidia-smi

In [ ]:
import pynvml

pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)

if device_name != b'Tesla T4':
    raise Exception("""
                    Unfortunately this instance does not have a T4 GPU.
                    Please make sure you've configured Colab to request 
                    a GPU instance type. 
                    Sometimes Colab allocates a Tesla K80 instead of a T4. 
                    Resetting the instance.
                    If you get a K80 GPU, try Runtime -> Reset all runtimes...
  """)
else:
    print('Great! You got the right kind of GPU!')

**Intall RAPIDS**

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

In [ ]:
from numba import cuda
print(cuda.gpus)

In [ ]:
!nvidia-smi

###  Average Calculations.

In [ ]:
import math

def average_sqrt(Mat, avg):
    """
        Average of the square root of all the entries of an array using loops.
    """
    avg = 0.
    for x in Mat:
        avg += math.sqrt(x)
    avg = avg/len(Mat)

In [ ]:
@njit
def average_sqrt_numba(Mat, avg):
    """
        Average of the square root of all the entries of an array using loops.
    """
    s = 0.
    for x in Mat:
        avg += math.sqrt(x)
    avg = avg/len(Mat)

We only need to specify the `cuda` target as argument of the decorator. 
Numba will automatically:

- Compile a Cuda kernel to execute the operations.
- Allocate GPU memory for the input.
- Execute the CUDA kernel with the correct kernel dimensions given the input sizes.
- Copy the result back from the GPU to the CPU.
- Return the result on the host.

In [ ]:
@jit(target ="cuda")
def average_sqrt_cuda(Mat, avg):
    """
        Average of the square root of all the entries of an array using loops.
    """
    avg = 0.
    for x in Mat:
        avg += math.sqrt(x)
    avg = avg/len(Mat)

In [ ]:
M = 1000000
Mat = np.random.rand(M)
avg = 0.

In [ ]:
time_reg = %timeit -o average_sqrt(Mat, avg)

In [ ]:
time_numba = %timeit -o average_sqrt_numba(Mat, avg)

In [ ]:
time_cuda1 = %timeit -o average_sqrt_cuda(Mat, avg)

In [ ]:
print("Regular Speedup: {}".format(time_reg.best/time_reg.best))
print("Numba   Speedup: {}".format(time_reg.best/time_numba.best))
print("Cuda    Speedup: {}".format(time_reg.best/time_cuda1.best))

### Writing Cuda Kernels

- In CUDA, the code you write will be executed by multiple threads at once (often hundreds or thousands). 
- The CUDA programming model allows you to abstract the GPU hardware into a software model defined by a thread hierachy that is composed of a **grid** containing **blocks** of **threads**. 
- These **threads** are the smallest individual unit in the programming model, and they execute together in groups (traditionally called **warps**, consisting of 32 threads each). 

### Kernel Declaration
A kernel function is a GPU function that is meant to be called from CPU code. It has two fundamental characteristics:

- Kernels cannot explicitly return a value; all result data must be written to an array passed to the function (if computing a scalar, you will probably pass a one-element array).
- Kernels explicitly declare their thread hierarchy when called: i.e. the number of thread blocks and the number of threads per block (note that while a kernel is compiled once, it can be called multiple times with different block sizes or grid sizes).

```python
@cuda.jit
def my_kernel(io_array):
    """
    Code for kernel.
    """
    # code here
```

### Kernel Invocation

There are two main steps:

- Instantiate the kernel proper, by specifying a number of blocks per grid and a number of threads per block. 
    - The product of the two will give the total number of threads launched. 
    - Kernel instantiation is done by taking the compiled kernel function and indexing it with a tuple of integers.
- Running the kernel, by passing it the input array (and any separate output arrays if necessary). 
    - By default, running a kernel is synchronous: the function returns when the kernel has finished executing and the data is synchronized back.

A kernel is typically launched in the following way:

```python
import numpy

# Create the data array - usually initialized some other way
data = numpy.ones(256)

# Set the number of threads in a block
threadsperblock = 32 

# Calculate the number of thread blocks in the grid
blockspergrid = (data.size + (threadsperblock - 1)) // threadsperblock

# Now start the kernel
my_kernel[blockspergrid, threadsperblock](data)

# Print the result
print(data)
```


**Choosing the Block Size**

The two-level thread hierarchy is important for the following reasons:

- On the software side, the block size determines how many threads share a given area of shared memory.
- On the hardware side, the block size must be large enough for full occupation of execution units; recommendations can be found in the CUDA C Programming Guide.

The block size you choose depends on a range of factors, including:

- The size of the data array
- The size of the shared mempory per block (e.g. 64KB)
- The maximum number of threads per block supported by the hardware (e.g. 512 or 1024)
- The maximum number of threads per multiprocessor (MP) (e.g. 2048)
- The maximum number of blocks per MP (e.g. 32)
- The number of threads that can be executed concurrently (a “warp” i.e. 32).


Determiming the best size for your grid of thread blocks is a complicated problem that often depends on the specific algorithm and hardware you're using. Here a few good rules of thumb:

- The size of a block should be a multiple of 32 threads, with typical block sizes between 128 and 512 threads per block.
- The size of the grid should ensure the full GPU is utilized where possible. Launching a grid where the number of blocks is 2x-4x the number of **streaming multiprocessors** on the GPU is a good starting place. (The Tesla K80 GPUs provided by Colaboratory have 15 SMs - more modern GPUs like the P100s on TigerGPU have 60+.)
- The CUDA kernel launch overhead does depend on the number of blocks, so it may not be best to launch a grid where the number of threads equals the number of input elements when the input size is very big. We'll show a pattern for dealing with large inputs below.

### Recommendations
To achieve the bes performance:

- Find ways to parallelize sequential code
- Minimize data transfers between the host and the device
- Adjust kernel launch configuration to maximize device utilization
- Ensure global memory accesses are coalesced
- Minimize redundant accesses to global memory whenever possible
- Avoid different execution paths within the same warp

In [ ]:
@cuda.jit
def average_sqrt_cuda_kernel(x, avg):
    tidx = cuda.threadIdx.x # this is the unique thread ID within a 1D block
    bidx = cuda.blockIdx.x  # Similarly, this is the unique block ID within the 1D grid

    block_dimx = cuda.blockDim.x  # number of threads per block
    grid_dimx = cuda.gridDim.x    # number of blocks in the grid
    
    start = tidx + bidx * block_dimx
    stride = block_dimx * grid_dimx
    
    avg = 0.
    for i in range(start, x.shape[0], stride):
        avg += np.sqrt(x[i])
    avg = avg/x.shape[0]

In [ ]:
threads_per_block = 128
blocks_per_grid = 30

In [ ]:
time_cuda2 = %timeit -o average_sqrt_cuda_kernel([blocks_per_grid, threads_per_block](Mat, avg)

In [ ]:
print("Cuda Kernel Speedup: {}".format(time_reg.best/time_cuda2.best))

### Matrix Multiplication

In [ ]:
N = 200
A = np.random.rand(N, N)
B = np.random.rand(N, N)
C = np.zeros_like(A)

In [ ]:
def matrix_multiplication(A, B, C):
    """
        Perform square matrix multiplication of C = A * B using loops.
    """
    n = len(A[0])
    for i in range(n):
        for j in range(n):
            tmp = 0.
            for k in range(n):
                tmp  += A[i, k]*B[k, j]
            C[i, j] = tmp

In [ ]:
tRegMat = %timeit -o matrix_multiplication(A, B, C)

In [ ]:
@njit(parallel=True)
def matrix_multiplication_numba(A, B, C):
    """
        Perform square matrix multiplication of C = A * B using loops.
    """
    n = len(A[0])
    for i in prange(n):
        for j in prange(n):
            tmp = 0.
            for k in prange(n):
                tmp += A[i, k]*B[k, j]
            C[i,j] = tmp

In [ ]:
tNumMat = %timeit -o matrix_multiplication_numba(A, B, C)

In [ ]:
@cuda.jit
def matrix_multiplication_cuda(A, B, C):
    """
      Perform square matrix multiplication of C = A * B using loops.
    """
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[row, k] * B[k, col]
        C[row, col] = tmp

In [ ]:
tCudaMat1 = %timeit -o matrix_multiplication_cuda(A, B, C)

In [ ]:
print("Speedup Numba : {}".format(tRegMat.best/tNumMat.best))
print("Speedup CUDA 1:  {}".format(tRegMat.best/tCudaMat1.best))

There can be a faster way for the matrix multiplication with Cuda:

- Each thread block is responsible for computing a square sub-matrix of C and each thread for computing an element of the sub-matrix. 
- The sub-matrix is equal to the product of a square sub-matrix of A (sA) and a square sub-matrix of B (sB). 
- In order to fit into the device resources, the two input matrices are divided into as many square sub-matrices of dimension TPB as necessary, and the result computed as the sum of the products of these square sub-matrices.
- Each product is performed by first loading sA and sB from global memory to shared memory, with one thread loading each element of each sub-matrix. 
- Once sA and sB have been loaded, each thread accumulates the result into a register (tmp). Once all the products have been calculated, the results are written to the matrix C in global memory.

In [ ]:
from numba import cuda, float32

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def matrix_multiplication_cuda_fast(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [ ]:
tCudaMat2 = %timeit -o matrix_multiplication_cuda_fast(A, B, C)

In [ ]:
print("Speedup Numba 2: {}".format(tRegMat.best/tNumMat2.best))
print("Speedup CUDA 1:  {}".format(tRegMat.best/tCudaMat1.best))
print("Speedup CUDA 2:  {}".format(tRegMat.best/tCudaMat2.best))

### Mandelbrot Fractal 

In [ ]:
import matplotlib.pyplot as plt
from timeit import default_timer as timer

**Pure Python**

In [ ]:
# color function for point at (x, y)
def mandel(x, y, max_iters):
    c = complex(x, y)
    z = 0.0j
    for i in range(max_iters):
        z = z*z + c
        if z.real*z.real + z.imag*z.imag >= 4:
            return i
    return max_iters

In [ ]:
def create_fractal(xmin, xmax, ymin, ymax, image, iters):
    height, width = image.shape

    pixel_size_x = (xmax - xmin)/width
    pixel_size_y = (ymax - ymin)/height

    for x in range(width):
        real = xmin + x*pixel_size_x
        for y in range(height):
            imag = ymin + y*pixel_size_y
            color = mandel(real, imag, iters)
            image[y, x]  = color

In [ ]:
gimage = np.zeros((1024, 1536), dtype=np.uint8)
xmin, xmax, ymin, ymax = np.array([-2.0, 1.0, -1.0, 1.0]).astype('float32')
iters = 50

start = timer()
create_fractal(xmin, xmax, ymin, ymax, gimage, iters)
dt_py = timer() - start

print("Mandelbrot created on CPU in {} s".format(dt_py))
plt.imshow(gimage);

**Numba Code**

In [ ]:
mandel_numba = jit(nb.uint32(nb.float32, nb.float32, nb.uint32))(mandel)

In [ ]:
@jit
def create_fractal_numba(xmin, xmax, ymin, ymax, image, iters):
    height, width = image.shape

    pixel_size_x = (xmax - xmin)/width
    pixel_size_y = (ymax - ymin)/height

    for x in range(width):
        real = xmin + x*pixel_size_x
        for y in range(height):
            imag = ymin + y*pixel_size_y
            color = mandel_numba(real, imag, iters)
            image[y, x]  = color

In [ ]:
gimage = np.zeros((1024, 1536), dtype=np.uint8)
xmin, xmax, ymin, ymax = np.array([-2.0, 1.0, -1.0, 1.0]).astype('float32')
iters = 50

start = timer()
create_fractal_numba(xmin, xmax, ymin, ymax, gimage, iters)
dt_numba = timer() - start

print("Mandelbrot created on CPU in {} s".format(dt_numba))
plt.imshow(gimage);

**CUDA Code**

In [ ]:
mandel_gpu = cuda.jit(restype=nb.uint32, 
                      argtypes=[nb.float32, nb.float32, nb.uint32], 
                      device=True)(mandel)

In [ ]:
@cuda.jit(argtypes=[nb.float32, nb.float32, nb.float32, nb.float32, nb.uint8[:,:], nb.uint32])
def create_fractal_kernel(xmin, xmax, ymin, ymax, image, iters):
    height, width = image.shape

    pixel_size_x = (xmax - xmin)/width
    pixel_size_y = (ymax - ymin)/height

    startX, startY = cuda.grid(2)
    gridX = cuda.gridDim.x * cuda.blockDim.x # stride in x
    gridY = cuda.gridDim.y * cuda.blockDim.y # stride in y

    for x in range(startX, width, gridX):
        real = xmin + x*pixel_size_x
        for y in range(startY, height, gridY):
            imag = ymin + y*pixel_size_y
            color = mandel_gpu(real, imag, iters)
            image[y, x]  = color

In [ ]:
gimage = np.zeros((1024, 1536), dtype=np.uint8)
blockdim = (32, 8)
griddim = (32, 16)
xmin, xmax, ymin, ymax = np.array([-2.0, 1.0, -1.0, 1.0]).astype('float32')
iters = 50

start = timer()
d_image = cuda.to_device(gimage)
create_fractal_kernel[griddim, blockdim](xmin, xmax, ymin, ymax, d_image, iters)
d_image.to_host()
dt_cuda = timer() - start

print("Mandelbrot created on GPU in {} s".format(dt_cuda))
plt.imshow(gimage);

In [ ]:
print("Speedup CPU:    {}".format(dt_py/dt_py))
print("Speedup Numba:  {}".format(dt_py/dt_numba))
print("Speedup CUDA:   {}".format(dt_py/dt_cuda))

### [cuDF and Dask-cuDF](https://docs.rapids.ai/api/cudf/stable/10min.html)

In [ ]:
!pip install dask_cudf

In [ ]:
import cudf
import cupy as cp
import dask_cudf
import numpy as np

print('Dask cuDF Version:', dask_cudf.__version__)

Use `cudf` to create a dataframe and perform operations:

In [ ]:
num_rows = 100000
df = cudf.DataFrame({'X':np.random.randint(1000, size=num_rows),
                     'Y':np.random.randint(1000, size=num_rows)})
df

In [ ]:
def add_squares(df):
    return df.X**2 + df.Y**2

In [ ]:
%%time

df['add_squares'] = add_squares(df)

In [ ]:
df

Do the same as above using `dask_cudf`:

In [ ]:
ddf = dask_cudf.from_cudf(df, npartitions=4)
ddf.head()

In [ ]:
%%time

ddf['z'] = add_squares(ddf).compute()

In [ ]:
ddf.head()

**Time Series Data**

`DataFrames` supports `datetime` typed columns, which allow users to interact with and filter data based on specific timestamps.

In [ ]:
import datetime as dt
import pandas as pd

date_df = cudf.DataFrame()
date_df['date'] = pd.date_range('01/05/1980', periods=15000, freq='D')
date_df['value'] = cp.random.sample(len(date_df))
date_df

In [ ]:
search_date1 = dt.datetime.strptime('2001-09-11', '%Y-%m-%d')
search_date2 = dt.datetime.strptime('2019-11-23', '%Y-%m-%d')

In [ ]:
%%time

date_df.query('date >= @search_date1' and 'date <= @search_date2')

In [ ]:
date_ddf = dask_cudf.from_cudf(date_df, npartitions=4)

In [ ]:
date_ddf.head()

In [ ]:
%%time

date_ddf.query('date >= @search_date1' and 'date <= @search_date2', 
               local_dict={'search_date1':search_date1, 
                           'search_date2':search_date2}).compute()